In [1]:
%load_ext autoreload
%autoreload 2

# TODO: get_class_weights_from_list implementation.

In [2]:
from notebook_utils import load_data, train, accuracy_multiclass
from notebook_utils import NN
from torchsummary import summary

import torch
import numpy as np

In [3]:
n_classes = 2

In [4]:
train_loader, test_data, test_labels = load_data(n_classes=n_classes)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
canonical = NN(n_classes=n_classes)
optimizer = torch.optim.Adam(canonical.parameters(), lr=1e-2)
loss_fn = torch.nn.BCEWithLogitsLoss()
print(summary(canonical, input_size=(3, 32, 32)))
# train(canonical, optimizer, loss_fn, train_loader, test_data, test_labels)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
              ReLU-2           [-1, 16, 32, 32]               0
       BatchNorm2d-3           [-1, 16, 32, 32]              32
            Conv2d-4           [-1, 16, 32, 32]           2,320
              ReLU-5           [-1, 16, 32, 32]               0
       BatchNorm2d-6           [-1, 16, 32, 32]              32
         MaxPool2d-7           [-1, 16, 16, 16]               0
           Dropout-8           [-1, 16, 16, 16]               0
            Conv2d-9           [-1, 32, 16, 16]           4,640
             ReLU-10           [-1, 32, 16, 16]               0
      BatchNorm2d-11           [-1, 32, 16, 16]              64
           Conv2d-12           [-1, 32, 16, 16]           9,248
             ReLU-13           [-1, 32, 16, 16]               0
      BatchNorm2d-14           [-1, 32,

In [6]:
canonical_state_dict = canonical.fe.state_dict()

In [7]:
overconfident = NN(n_classes=n_classes)

for name, parameter in overconfident.fe.named_parameters():
    parameter = canonical_state_dict[name]
    parameter.requires_grad = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
overconfident_weight = torch.Tensor([2]).to(device)
    
optimizer = torch.optim.Adam(overconfident.parameters(), lr=1e-2)
loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=overconfident_weight)
train(overconfident, optimizer, loss_fn, train_loader, test_data, test_labels)

RuntimeError: expected backend CUDA and dtype Float but got backend CPU and dtype Float

In [ ]:
underconfident = NN(n_classes=n_classes)

for name, parameter in underconfident.fe.named_parameters():
    parameter = canonical_state_dict[name]
    parameter.requires_grad = False

optimizer = torch.optim.Adam(underconfident.parameters(), lr=1e-2)
loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=0.5)
train(underconfident, optimizer, loss_fn, train_loader, test_data, test_labels)